In [2]:
import datetime
import backtrader as bt
import os.path as path

class TestStrategy(bt.Strategy):

  def log(self, txt, dt=None):
    dt = dt or self.datas[0].datetime.date(0)
    print('%s, %s' % (dt.isoformat(), txt))

  def __init__(self):
    self.dataclose = self.datas[0].close
    self.order = None

  def notify_order(self, order):
    if order.status in [order.Submitted, order.Accepted]:
      return
    
    if order.status in [order.Completed]:
      if order.isbuy():
        self.log('已买入, %.2f' % order.executed.price)
      elif order.issell():
        self.log('已卖出, %.2f' % order.executed.price)

      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]:
      self.log('订单取消/保证金不足/拒绝')
    
    self.order = None

  
  def next(self):
    self.log('Close, %.2f' % self.dataclose[0])

    if self.order:
      return

    if not self.position:
      if self.dataclose[0] < self.dataclose[-1]:
        if self.dataclose[-1] < self.dataclose[-2]:
          self.log('买入, %.2f' % self.dataclose[0])
          self.order = self.buy()
    else:
      if len(self) >= (self.bar_executed + 5):
        self.log('卖出, %.2f' % self.dataclose[0])
        self.order = self.sell()

if __name__ == '__main__':
  cerebro = bt.Cerebro()

  datapath = path.join(path.abspath('.'), '../10/orcl-1995-2014.txt')
  data = bt.feeds.YahooFinanceCSVData(
    dataname=datapath,
    fromdate=datetime.datetime(2000, 1, 1),
    todate=datetime.datetime(2000, 12, 31),
    reverse=False,
  )

  cerebro.adddata(data)
  cerebro.addstrategy(TestStrategy)
  cerebro.broker.setcash(100000)

  cerebro.run()

  print('资金: %.2f' % cerebro.broker.getvalue())

2000-01-03, Close, 26.27
2000-01-04, Close, 23.95
2000-01-05, Close, 22.68
2000-01-05, 买入, 22.68
2000-01-06, 已买入, 22.27
2000-01-06, Close, 21.35
2000-01-07, Close, 22.99
2000-01-10, Close, 25.74
2000-01-11, Close, 24.99
2000-01-12, Close, 23.49
2000-01-13, Close, 23.36
2000-01-13, 卖出, 23.36
2000-01-14, 已卖出, 24.24
2000-01-14, Close, 23.75
2000-01-18, Close, 24.74
2000-01-19, Close, 25.41
2000-01-20, Close, 26.35
2000-01-21, Close, 26.55
2000-01-24, Close, 24.10
2000-01-25, Close, 25.10
2000-01-26, Close, 24.49
2000-01-27, Close, 23.04
2000-01-27, 买入, 23.04
2000-01-28, 已买入, 22.90
2000-01-28, Close, 21.07
2000-01-31, Close, 22.22
2000-02-01, Close, 24.02
2000-02-02, Close, 24.16
2000-02-03, Close, 25.21
2000-02-04, Close, 25.71
2000-02-04, 卖出, 25.71
2000-02-07, 已卖出, 26.38
2000-02-07, Close, 26.66
2000-02-08, Close, 26.49
2000-02-09, Close, 26.66
2000-02-10, Close, 27.71
2000-02-11, Close, 26.55
2000-02-14, Close, 27.66
2000-02-15, Close, 27.30
2000-02-16, Close, 27.24
2000-02-16, 买入, 27.2